In [45]:
# Notebook vorbereiten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
ab_ag_raw = pd.read_csv('data/ab_ag.tsv', sep='\t')
columns = pd.read_csv('data/columns.tsv', sep='\t')
uniprot_data = pd.read_csv('data/uniprot_data.tsv', sep='\t')

# ab_ag aus ab_ag_raw aufbereiten
ab_ag = ab_ag_raw[["pdb", "Hchain", "Lchain", "antigen_chain", "antigen_name", "antigen_species"]]
# Füge Spalten für die CDR-Sequenzen in ab_ag hinzu
ab_ag["CDR_H1"] = np.nan
ab_ag["CDR_H2"] = np.nan
ab_ag["CDR_H3"] = np.nan
ab_ag["CDR_L1"] = np.nan
ab_ag["CDR_L2"] = np.nan
ab_ag["CDR_L3"] = np.nan
# Entferne Einträge ohne Antigennamen
ab_ag.dropna(subset=["antigen_chain"], inplace=True)

# Ich hab aus Versehen das Code-Feld gelöscht! Code von Anna für doppelte pdb Einträge zurückholen!

C:\Users\marce\AppData\Local\Temp\ipykernel_6880\647701685.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_ag["H1_seq"] = np.nan
C:\Users\marce\AppData\Local\Temp\ipykernel_6880\647701685.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_ag["H2_seq"] = np.nan
C:\Users\marce\AppData\Local\Temp\ipykernel_6880\647701685.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [ ]:
# Funktionen aus dem sabdab_downloader importieren
from common.sabdab_downloader import getchothpdb
from common.sabdab_downloader import getannotation

# Erstelle eine Liste mit allen eindeutigen PDB-Einträgen aus ab_ag
pdb_entries = ab_ag["pdb"].unique()

# Lade die Chothia-Daten und Chothia-nummerierten Sequenzen für jeden PDB-Eintrag herunter und speichere sie im Ordner des entsprechenden PDB-Eintrags
for pdb in pdb_entries:
    # Zeilenweise Einträge für den aktuellen PDB-Eintrag extrahieren (für die Chothia-annotierten Sequenzen)
    fab_list = ab_ag_raw[ab_ag_raw["pdb"] == pdb].values.tolist()
    # Erstelle die Ordner für Chothia-Daten und Chothia-annotierte Sequenz für den PDB-Eintrag, falls er nicht existiert
    if not os.path.exists(f"data/sabdab_downloads/{pdb}/chothia_file"):
        os.makedirs(f"data/sabdab_downloads/{pdb}/chothia_file")
    if not os.path.exists(f"data/sabdab_downloads/{pdb}/chothia_sequence"):
        os.makedirs(f"data/sabdab_downloads/{pdb}/chothia_sequence")
    # Lade die Chothia-Daten und Chothia-nummerierten Sequenzen herunter oder gib eine Fehlermeldung für entsprechende PDB-Einträge aus
    try:
        getchothpdb(pdb, f"data/sabdab_downloads/{pdb}/chothia_file")
    except Exception as e:
        print(f"Could not download Chothia-File for {pdb}: {e}")
    try:
        getannotation(pdb, fab_list, f"data/sabdab_downloads/{pdb}/chothia_sequence")
    except Exception as e:
        print(f"Could not download Chothia-annotated sequence for {pdb}: {e}")

In [67]:
# Überarbeiteter Code zum Herunterladen der Antikörpersequenzen. Direkt in ab_ag_raw speichern
# Funktionen aus dem sabdab_downloader importieren
from common.sabdab_downloader import getchothpdb
from common.sabdab_downloader import getannotation

# Definiere CDR-Regionen als slices für die H- und L-Ketten (Literatur überprüfen, ob die Indizes korrekt sind!)
cdr_regions = {
        "H1": slice(26, 32),
        "H2": slice(52, 56),
        "H3": slice(95, 102),
        "L1": slice(24, 34),
        "L2": slice(50, 56),
        "L3": slice(89, 97)
}

# Erstelle eine Liste mit allen PDB-Einträgen aus ab_ag (enthält Duplikate)
pdb_entries = ab_ag["pdb"].tolist()

# Lade die Chothia-Daten und Chothia-nummerierten Sequenzen für jeden PDB-Eintrag herunter und speichere sie im Ordner des entsprechenden PDB-Eintrags
for x in range(0,6): # Testlauf, nur die ersten 5 PDB-Einträge
    # Zeilenweise Einträge für den aktuellen PDB-Eintrag extrahieren (für die Chothia-annotierten Sequenzen)
    fab_list = ab_ag.iloc[x,:].values.tolist()
    pdb = fab_list[0] # PDB-ID aus der Liste extrahieren
    # Erstelle den Ordner für die Chothia-annotierte Sequenz des PDB-Eintrags, falls er nicht existiert
    if not os.path.exists(f"data/sabdab_download/{pdb}"):
        os.makedirs(f"data/sabdab_download/{pdb}")
    # Lade die Chothia-nummerierten Sequenzen herunter oder gib eine Fehlermeldung für entsprechende PDB-Einträge aus
    try:
        getannotation(pdb, fab_list, f"data/sabdab_download/{pdb}")
        # Extrahiere die CDR-Sequenzen aus den heruntergeladenen Chothia-annotierten Sequenzen und speichere sie in ab_ag
        # Speichere jede .ann-Datei in einm DataFrame
        VH_chothia_seq = pd.read_csv(f"data/sabdab_download/{pdb}/{pdb}_{fab_list[1]}_VH.ann", sep='\t', header=None, names=['Chothia_Position', 'Aminosäure'])
        VL_chothia_seq = pd.read_csv(f"data/sabdab_download/{pdb}/{pdb}_{fab_list[2]}_VL.ann", sep='\t', header=None, names=['Chothia_Position', 'Aminosäure'])
        # Extrahiere die CDR-Sequenzen aus den Chothia-annotierten Sequenzen
        H1_seq = VH_chothia_seq.loc[cdr_regions["H1"], "Aminosäure"].tolist()
        ab_ag.loc[ab_ag["pdb"] == pdb, "CDR_H1"] = H1_seq
        H2_seq = VH_chothia_seq.loc[cdr_regions["H2"], "Aminosäure"].tolist()
        ab_ag.loc[ab_ag["pdb"] == pdb, "CDR_H2"] = H1_seq
        H3_seq = VH_chothia_seq.loc[cdr_regions["H3"], "Aminosäure"].tolist()
        ab_ag.loc[ab_ag["pdb"] == pdb, "CDR_H3"] = H1_seq
        L1_seq = VL_chothia_seq.loc[cdr_regions["L1"], "Aminosäure"].tolist()
        ab_ag.loc[ab_ag["pdb"] == pdb, "CDR_L1"] = H1_seq
        L2_seq = VL_chothia_seq.loc[cdr_regions["L2"], "Aminosäure"].tolist()
        ab_ag.loc[ab_ag["pdb"] == pdb, "CDR_L2"] = H1_seq
        L3_seq = VL_chothia_seq.loc[cdr_regions["L3"], "Aminosäure"].tolist()
        ab_ag.loc[ab_ag["pdb"] == pdb, "CDR_L3"] = H1_seq
    except Exception as e:
        print(f"Could not download Chothia-annotated sequence for {pdb}: {e}")
    

Could not download Chothia-annotated sequence for 9ds2: HTTP Error 404: Not Found
Could not download Chothia-annotated sequence for 9ds2: HTTP Error 404: Not Found
Could not download Chothia-annotated sequence for 9ds2: HTTP Error 404: Not Found
Could not download Chothia-annotated sequence for 8uzp: HTTP Error 404: Not Found
Could not download Chothia-annotated sequence for 8uzp: HTTP Error 404: Not Found
Could not download Chothia-annotated sequence for 8veb: HTTP Error 404: Not Found
